In [3]:
import tensorflow as tf

In [ ]:
max_tokens = 0
dimension = 0


drop_out_rate = 0.1
learning_rate = 0.001
number_of_epochs = 100
number_of_classes = 2
batch_size = 32
optimizer = optimizers.Adam(learning_rate, learning_rate / 100)
input_tensor = Input(shape=input_shape)
metrics = ["accuracy"]

x = layers.Conv2D(16, (3,3), activation="relu", padding="same")(input_tensor)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D(maxpool_size)(x)
x = layers.Dropout(rate=drop_out_rate)(x)

x = layers.Conv2D(32, (3,3), activation="relu", padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D(maxpool_size)(x)
x = layers.Dropout(rate=drop_out_rate)(x)

x = layers.Conv2D(64, (3,3), activation="relu", padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D(maxpool_size)(x)
x = layers.Dropout(rate=drop_out_rate)(x)

x = layers.Conv2D(256, (3,3), activation="relu", padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.GlobalMaxPool2D()(x)
    x = layers.Dropout(rate=(drop_out_rate * 2))(x) # Increasing drop-out rate here to prevent overfitting

    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dense(1028, activation="relu")(x)
    output_tensor = layers.Dense(2, activation="softmax")(x)

    model = tf.keras.Model(input_tensor, output_tensor)
    model.compile(optimizer=optimizer, loss=keras.losses.binary_crossentropy, metrics=metrics)

# Variable-length int sequences.
query_input = tf.keras.Input(shape=(None,), dtype='int32')
value_input = tf.keras.Input(shape=(None,), dtype='int32')

# Embedding lookup.
token_embedding = tf.keras.layers.Embedding(max_tokens, dimension)
# Query embeddings of shape [batch_size, Tq, dimension].
query_embeddings = token_embedding(query_input)
# Value embeddings of shape [batch_size, Tv, dimension].
value_embeddings = token_embedding(query_input)

# CNN layer.
cnn_layer = tf.keras.layers.Conv1D(
    filters=100,
    kernel_size=4,
    # Use 'same' padding so outputs have the same shape as inputs.
    padding='same')
# Query encoding of shape [batch_size, Tq, filters].
query_seq_encoding = cnn_layer(query_embeddings)
# Value encoding of shape [batch_size, Tv, filters].
value_seq_encoding = cnn_layer(value_embeddings)

# Query-value attention of shape [batch_size, Tq, filters].
query_value_attention_seq = tf.keras.layers.Attention()(
    [query_seq_encoding, value_seq_encoding])

# Reduce over the sequence axis to produce encodings of shape
# [batch_size, filters].
query_encoding = tf.keras.layers.GlobalAveragePooling1D()(
    query_seq_encoding)
query_value_attention = tf.keras.layers.GlobalAveragePooling1D()(
    query_value_attention_seq)

# Concatenate query and document encodings to produce a DNN input layer.
input_layer = tf.keras.layers.Concatenate()(
    [query_encoding, query_value_attention])

# Add DNN layers, and create Model.
# ...